# Imports and Setup

In [ ]:
!pip install split-folders
!pip install xmltodict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import splitfolders
import numpy as np
from sklearn.metrics import classification_report
from torchvision import models

import xmltodict
import json
import cv2 as cv
from google.colab.patches import cv2_imshow
import sklearn.datasets as ds 
import glob, os
import pandas as pd
import matplotlib.pyplot as plt

For training and evaluation, Components_Multiclass.zip must be uploaded manually.
This was done, as loading the data directly from Google Drive took too long.

In [ ]:
!unzip 'Components_Multiclass.zip'

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
  inflating: Components_Multiclass/17_Schraube_abstand/Schraube_abstand_1286.jpg  
  inflating: __MACOSX/Components_Multiclass/17_Schraube_abstand/._Schraube_abstand_1286.jpg  
  inflating: Components_Multiclass/17_Schraube_abstand/Schraube_abstand_3863.jpg  
  inflating: __MACOSX/Components_Multiclass/17_Schraube_abstand/._Schraube_abstand_3863.jpg  
  inflating: Components_Multiclass/17_Schraube_abstand/Schraube_abstand_376.jpg  
  inflating: __MACOSX/Components_Multiclass/17_Schraube_abstand/._Schraube_abstand_376.jpg  
  inflating: Components_Multiclass/17_Schraube_abstand/Schraube_abstand_1655.jpg  
  inflating: __MACOSX/Components_Multiclass/17_Schraube_abstand/._Schraube_abstand_1655.jpg  
  inflating: Components_Multiclass/17_Schraube_abstand/Schraube_abstand_3724.jpg  
  inflating: __MACOSX/Components_Multiclass/17_Schraube_abstand/._Schraube_abstand_3724.jpg  
  inflating: Components_Multiclass/17_Schraube_abs

# Data Loading and Pre-Processing

In [ ]:
# Split data into train, test and validation sets
splitfolders.ratio("Components_Multiclass", output="output", seed=1337, ratio=(0.7, 0.2, 0.1))

# Create imagefolders and transform data
train_folder = torchvision.datasets.ImageFolder('/content/output/train',
                                                transform =  torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                                                             torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                                                                             torchvision.transforms.RandomHorizontalFlip(),
                                                                                             torchvision.transforms.RandomVerticalFlip(),
                                                                                             torchvision.transforms.Resize((244,244))]))

val_folder = torchvision.datasets.ImageFolder('/content/output/val',
                                                transform =  torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                                                             torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                                                                             torchvision.transforms.RandomHorizontalFlip(),
                                                                                             torchvision.transforms.RandomVerticalFlip(),
                                                                                             torchvision.transforms.Resize((244,244))]))

test_folder = torchvision.datasets.ImageFolder('/content/output/test',
                                                transform =  torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                                                             torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                                                                             torchvision.transforms.RandomHorizontalFlip(),
                                                                                             torchvision.transforms.RandomVerticalFlip(),
                                                                                             torchvision.transforms.Resize((244,244))]))

# Dataloaders
train_dataloader = torch.utils.data.DataLoader(train_folder, batch_size=32, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_folder, batch_size=32)
test_dataloader = torch.utils.data.DataLoader(test_folder, batch_size=32)

Copying files: 4762 files [00:01, 4355.07 files/s]


In [ ]:
# Label Dictionary to make evaluation more interpretable
label_dict = {
    0:'COM_male',
    1:'Display_Port',
    2:'Ethernet',
    3:'LED_HDD',
    4:'LED_PWR',
    5:'LED_SSD',
    6:'Line_out',
    7:'Loch',
    8:'LOGO_AllenBradley_front',
    9:'LOGO_AllenBradley_icon',
    10:'LOGO_Kontron_front',
    11:'LOGO_Kontron_icon',
    12:'Power_5V',
    13:'Powerstecker_off',
    14:'Powerstecker_on',
    15:'Reset_button',
    16:'Schraube',
    17:'Schraube_abstand',
    18:'Schraube_gespiegelt',
    19:'Schraube_halb',
    20:'Schraube_rund',
    21:'USB_2',
    22:'USB_3',
    23:'VGA'   
}

# Training Loop

In [ ]:
# Training Loop
def train_model(net, train_dataloader, val_dataloader):
  net = net.to('cuda')

  optimizer = torch.optim.Adam(net.parameters(), lr=3e-4)
  loss_fct = nn.CrossEntropyLoss()

  train_losses = []
  val_losses = []
  for t in range(10):
    net.train()

    epoch_train_loss = 0.
    for X_batch, y_batch in train_dataloader:
      X_batch = X_batch.to('cuda')
      y_batch = y_batch.to('cuda')

      y_pred = net(X_batch)

      loss = loss_fct(y_pred.squeeze(), y_batch)
      epoch_train_loss += loss

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    train_losses.append(epoch_train_loss.cpu().data / len(train_dataloader))

    net.eval()
    epoch_val_loss = 0.
    for X_batch, y_batch in val_dataloader:
      X_batch = X_batch.to('cuda')
      y_batch = y_batch.to('cuda')
      with torch.no_grad():
        y_pred_val = net(X_batch)
        loss = loss_fct(y_pred_val.squeeze(), y_batch )
        epoch_val_loss += loss
    val_losses.append(epoch_val_loss.cpu().data / len(val_dataloader))

    return net

# Eval

In [ ]:
def evaluate_model(trained_model, test_dataloader):

  trained_model.eval()
  y_pred_list = []
  y_true_list = []
  for X_batch, y_batch in test_dataloader:
    X_batch = X_batch.to('cuda')
    y_pred = trained_model(X_batch)

    batch_pred = y_pred.tolist()

    # Loop through each prediction in batch (list of lists) and append index of max value to y_pred_list
    for i in range(len(batch_pred)):
      y_pred_list.append(np.argmax(batch_pred[i]))

    y_true_list.extend(y_batch.tolist())

    # Substitue number labels with text labels to make report more interpretable

  pred = []
  true = []

  for i in range(len(y_pred_list)):
    pred.append(label_dict[y_pred_list[i]])
    true.append(label_dict[y_true_list[i]])

  print(classification_report(true, pred))

# Hand-crafted Model

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5)

        self.fc1 = nn.Linear(in_features=107648, out_features=50)  
        # 24 output features to accomodate for 24 different components
        self.fc2 = nn.Linear(in_features=50, out_features=24)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(len(x), 107648)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
net = Net()
trained_model = train_model(net, train_dataloader, val_dataloader)
evaluate_model(trained_model, test_dataloader)

                         precision    recall  f1-score   support

               COM_male       0.85      1.00      0.92        11
           Display_Port       1.00      0.57      0.73        21
               Ethernet       0.94      1.00      0.97        31
                LED_HDD       0.83      0.83      0.83         6
                LED_PWR       0.92      0.57      0.71        21
                LED_SSD       1.00      1.00      1.00         6
LOGO_AllenBradley_front       1.00      0.90      0.95        10
 LOGO_AllenBradley_icon       1.00      0.90      0.95        10
     LOGO_Kontron_front       1.00      1.00      1.00        11
      LOGO_Kontron_icon       1.00      1.00      1.00         6
               Line_out       1.00      1.00      1.00        10
                   Loch       0.00      0.00      0.00         8
               Power_5V       1.00      1.00      1.00        10
       Powerstecker_off       1.00      0.71      0.83         7
        Powerstecker_on 

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Pre-trained Model (ResNet)

In [ ]:
net = models.resnet18(pretrained=True)
# change output layer to 24 to accomodate for 24 different components
net.fc = nn.Linear(512, 24)

trained_model = train_model(net, train_dataloader, val_dataloader)
evaluate_model(trained_model, test_dataloader)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

                         precision    recall  f1-score   support

               COM_male       1.00      1.00      1.00        11
           Display_Port       1.00      1.00      1.00        21
               Ethernet       1.00      1.00      1.00        31
                LED_HDD       1.00      1.00      1.00         6
                LED_PWR       1.00      1.00      1.00        21
                LED_SSD       1.00      1.00      1.00         6
LOGO_AllenBradley_front       1.00      1.00      1.00        10
 LOGO_AllenBradley_icon       1.00      1.00      1.00        10
     LOGO_Kontron_front       1.00      1.00      1.00        11
      LOGO_Kontron_icon       1.00      1.00      1.00         6
               Line_out       1.00      1.00      1.00        10
                   Loch       0.89      1.00      0.94         8
               Power_5V       1.00      1.00      1.00        10
       Powerstecker_off       1.00      1.00      1.00         7
        Powerstecker_on 